In [1]:
# CS544 p2
# Xing Hong xhong37@wisc.edu
# Xingke Sun xsun326@wisc.edu

In [2]:
import pandas as pd
import torch

In [3]:
train_doc = pd.read_csv("train.csv")
test_doc = pd.read_csv("test.csv")

In [4]:
trainX = torch.tensor(train_doc.values[:, :-1], dtype=torch.float64)
trainY = torch.tensor(train_doc.values[:, -1:], dtype=torch.float64)
testX = torch.tensor(test_doc.values[:, :-1], dtype=torch.float64)
testY = torch.tensor(test_doc.values[:, -1:], dtype=torch.float64)

In [5]:
#Q1
consume_bytes_trainX = trainX.element_size() * trainX.nelement()
int(consume_bytes_trainX)

83520

In [6]:
#Q2
trainX_initial_float16 = trainX.to(dtype=torch.float16)
trainX_change_float64 = trainX_initial_float16.to(dtype=torch.float64)
biggest_diff = (trainX_initial_float16 - trainX_change_float64).abs().max().item()
biggest_diff

0.0

In [7]:
#Q3
c_available = torch.cuda.is_available()
c_available

False

In [8]:
coef = torch.tensor([
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040], # POS_50_59_CP
        [0.0300], # POS_60_69_CP
        [0.0300],
        [0.0300],
        [0.0300]
], dtype=trainX.dtype)
coef

tensor([[0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0040],
        [0.0300],
        [0.0300],
        [0.0300],
        [0.0300]], dtype=torch.float64)

In [9]:
#Q4
first_number_tract_predict = (testX[0] @ coef).item()
first_number_tract_predict

9.844

In [10]:
#Q5
average_predict_number = (testX @ coef).mean().item()
average_predict_number

12.073632183908048

In [11]:
#Q6
x = torch.tensor(0.0, requires_grad=True)
y = x**2 - 8*x +19
y.item()

19.0

In [12]:
#Q7
optimizer = torch.optim.SGD([x], lr=0.1)
for n in range(100):
    y = x**2 - 8*x +19
    y.backward()
    optimizer.step()
    optimizer.zero_grad()
x.item()

3.999999523162842

In [13]:
#Q8
coef = torch.zeros((trainX.shape[1], 1), dtype = torch.float64, requires_grad=True)

prediction_mse = trainX @ coef
MSE = (((prediction_mse - trainY)**2).sum()/ trainY.shape[0]).item()
MSE

197.8007662835249

In [14]:
#Q9
torch.manual_seed(544)
ds = torch.utils.data.TensorDataset(trainX, trainY)
dl = torch.utils.data.DataLoader(ds, batch_size=50, shuffle=True)

loss_function = torch.nn.MSELoss()
optimizer  = torch.optim.SGD([coef],lr = 0.000002)

for n in range(500):
    for X,Y in dl:
        prediction = X @ coef
        loss = loss_function(prediction, Y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

prediction_training = trainX @ coef
prediction_training_MSE = loss_function(prediction_training,trainY).item()
prediction_training_MSE

26.8113940147193

In [15]:
#Q10
prediction_over_test = testX @ coef
MSE_over_test = loss_function(prediction_over_test,testY).item()
MSE_over_test

29.05854692548551